### Meeting a Sayed Athar's request, I'm using the Kernel altered by Khoi Nguyen to explain how the whole code works.
### If any part is not clear, please comment.  
### Please upvote if it was helpful.

In [1]:
import pandas as pd
import pyarrow.parquet as pq # Used to read the data
import os 
import numpy as np
from keras.layers import * # Keras is the most friendly Neural Network library, this Kernel use a lot of layers classes
from keras.models import Model
from tqdm import tqdm # Processing time measurement
from sklearn.model_selection import train_test_split 
from keras import backend as K # The backend give us access to tensorflow operations and allow us to create the Attention class
from keras import optimizers # Allow us to access the Adam class to modify some parameters
from sklearn.model_selection import GridSearchCV, StratifiedKFold # Used to use Kfold to train our model
from keras.callbacks import * # This object helps the model to train in a smarter way, avoiding overfitting

from scipy.signal import chirp, find_peaks, peak_widths
import pywt

from multiprocessing import Pool
from sklearn.preprocessing import MinMaxScaler

import warnings

warnings.filterwarnings('ignore')

/home/cwademo123/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# select how many folds will be created
N_SPLITS = 5
# it is just a constant with the measurements data size
sample_size = 800000

In [3]:
def matthews_correlation(y_true, y_pred):
    '''Calculates the Matthews correlation coefficient measure for quality
    of binary classification problems.
    '''
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)

    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)

    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    return numerator / (denominator + K.epsilon())

In [4]:
# https://www.kaggle.com/suicaokhoailang/lstm-attention-baseline-0-652-lb

class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [5]:
# just load train data
df_train = pd.read_csv('../input/metadata_train.csv')
# set index, it makes the data access much faster
df_train = df_train.set_index(['id_measurement', 'phase'])
df_train.head()

signal_id  target
id_measurement phase                   
0              0              0       0
               1              1       0
               2              2       0
1              0              3       1
               1              4       1

In [6]:
# in other notebook I have extracted the min and max values from the train data, the measurements
max_num = 127
min_num = -128

In [7]:
# This function standardize the data from (-128 to 127) to (-1 to 1)
# Theoretically it helps in the NN Model training, but I didn't tested without it
def min_max_transf(ts, min_data, max_data, range_needed=(-1,1)):
    if min_data < 0:
        ts_std = (ts + abs(min_data)) / (max_data + abs(min_data))
    else:
        ts_std = (ts - min_data) / (max_data - min_data)
    if range_needed[0] < 0:    
        return ts_std * (range_needed[1] + abs(range_needed[0])) + range_needed[0]
    else:
        return ts_std * (range_needed[1] - range_needed[0]) + range_needed[0]

In [8]:
def maddest(d, axis=None):
    """
    Mean Absolute Deviation
    """
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

In [9]:
def denoise_signal( x, wavelet='db4', level=1):
    """
    1. Adapted from waveletSmooth function found here:
    http://connor-johnson.com/2016/01/24/using-pywavelets-to-remove-high-frequency-noise/
    2. Threshold equation and using hard mode in threshold as mentioned
    in section '3.2 denoising based on optimized singular values' from paper by Tomas Vantuch:
    http://dspace.vsb.cz/bitstream/handle/10084/133114/VAN431_FEI_P1807_1801V001_2018.pdf
    """
    
    # Decompose to get the wavelet coefficients
    coeff = pywt.wavedec( x, wavelet, mode="per", level=level)
    
    # Calculate sigma for threshold as defined in http://dspace.vsb.cz/bitstream/handle/10084/133114/VAN431_FEI_P1807_1801V001_2018.pdf
    # As noted by @harshit92 MAD referred to in the paper is Mean Absolute Deviation not Median Absolute Deviation
    sigma = (1/0.6745) * maddest( coeff[-level] )

    # Calculte the univeral threshold
    uthresh = sigma * np.sqrt( 2*np.log( len( x ) ) )
    coeff[1:] = ( pywt.threshold( i, value=uthresh, mode='hard' ) for i in coeff[1:] )
    
    # Reconstruct the signal using the thresholded coefficients
    return pywt.waverec( coeff[1:], wavelet, mode='per' )

In [10]:
def remove_corona(x_dn, maxDistance=10, maxHeightRatio=0.25, maxTicksRemoval=500):
    index = pd.Series(x_dn).loc[np.abs(x_dn)>0].index
    corona_idx = []
    for idx in index:
        for i in range(1,maxDistance+1):
            if idx+i < pd.Series(x_dn).shape[0]:
                if x_dn[idx+i]/(x_dn[idx]+1e-04)<-maxHeightRatio:
                    x_dn[idx:idx+maxTicksRemoval] = 0
                    corona_idx.append(idx)
    return x_dn, corona_idx

In [11]:
# This is one of the most important peace of code of this Kernel
# Any power line contain 3 phases of 800000 measurements, or 2.4 millions data 
# It would be praticaly impossible to build a NN with an input of that size
# The ideia here is to reduce it each phase to a matrix of <n_dim> bins by n features
# Each bean is a set of 5000 measurements (800000 / 160), so the features are extracted from this 5000 chunk data.
def transform_ts(ts, n_dim=160, min_max=(-1,1)):
    # convert data into -1 to 1
    ts_std = min_max_transf(ts, min_data=min_num, max_data=max_num)
    # bucket or chunk size, 5000 in this case (800000 / 160)
    bucket_size = int(sample_size / n_dim)
    # new_ts will be the container of the new data
    ts_wave = denoise_signal(ts_std, wavelet='haar', level=1)
    ts_rm, corona_idx = remove_corona(ts_wave)
    
    new_ts = []
    # this for iteract any chunk/bucket until reach the whole sample_size (800000)
    for i in range(0, sample_size, bucket_size):
        # cut each bucket to ts_range
        ts_range = ts_std[i:i + bucket_size]
        ts_rm_range = ts_rm[int(i/2):int(i/2) + int(bucket_size/2)]
        #ts_rm_range = pd.Series(ts_rm_range)
        
        # calculate each feature
        mean = ts_range.mean()
        std = ts_range.std() # standard deviation
        std_top = mean + std # I have to test it more, but is is like a band
        std_bot = mean - std
        # I think that the percentiles are very important, it is like a distribuiton analysis from eath chunk
        percentil_calc = np.percentile(ts_range, [0, 1, 25, 50, 75, 99, 100]) 
        max_range = percentil_calc[-1] - percentil_calc[0] # this is the amplitude of the chunk
        relative_percentile = percentil_calc - mean # maybe it could heap to understand the asymmetry
        
        #numpeaks = ts_rm_range[ts_rm_range!=0].count()
        numpeaks = np.sum(ts_rm_range!=0)
        #numpospeaks = ts_rm_range[ts_rm_range>0].count()
        numpospeaks = np.sum(ts_rm_range>0)
        #numnegpeaks = ts_rm_range[ts_rm_range<0].count()
        numnegpeaks = np.sum(ts_rm_range<0)

        meanamp = np.mean(ts_rm_range)
        meanamppos = np.mean(ts_rm_range[ts_rm_range>0])
        meanampneg = np.mean(ts_rm_range[ts_rm_range<0])

        maxamp = np.max(ts_rm_range)
        minamp = np.min(ts_rm_range)
        
        peaks, _ = find_peaks(ts_rm_range)
        results_full = peak_widths(ts_rm_range, peaks, rel_height=1)
        
        if len(results_full[0])==0:
            maxwidth = 0
            minwidth = 0
        else:
            maxwidth = np.max(results_full[0])
            minwidth = np.min(results_full[0])
        
        
        
        feat_array = np.asarray([mean, std, std_top, std_bot, max_range])
        feat_array_2 = np.asarray([numpeaks, numpospeaks, numnegpeaks, meanamp, meanamppos, meanampneg,
                                 maxamp, minamp, maxwidth, minwidth])
        
        new_ts.append(np.concatenate([feat_array, feat_array_2, percentil_calc, relative_percentile]))
        
    scaler = MinMaxScaler(feature_range=(-1, 1))
    new_ts = np.asarray(new_ts)
    new_ts[np.isnan(new_ts)] = 0
    
    new_ts = scaler.fit_transform(new_ts)
        
    return new_ts

In [12]:
# this function take a piece of data and convert using transform_ts(), but it does to each of the 3 phases
# if we would try to do in one time, could exceed the RAM Memmory
def prep_data(start, end):
    # load a piece of data from file
    praq_train = pq.read_pandas('../input/train.parquet', columns=[str(i) for i in range(start, end)]).to_pandas()
    X = []
    y = []
    # using tdqm to evaluate processing time
    # takes each index from df_train and iteract it from start to end
    # it is divided by 3 because for each id_measurement there are 3 id_signal, and the start/end parameters are id_signal
    for id_measurement in tqdm(df_train.index.levels[0].unique()[int(start/3):int(end/3)]):
        X_signal = []
        # for each phase of the signal
        for phase in [0,1,2]:
            # extract from df_train both signal_id and target to compose the new data sets
            signal_id, target = df_train.loc[id_measurement].loc[phase]
            # but just append the target one time, to not triplicate it
            if phase == 0:
                y.append(target)
            # extract and transform data into sets of features
            X_signal.append(transform_ts(praq_train[str(signal_id)]))
        # concatenate all the 3 phases in one matrix
        X_signal = np.concatenate(X_signal, axis=1)
        # add the data to X
        X.append(X_signal)
    X = np.asarray(X)
    y = np.asarray(y)
    return X, y

In [13]:
def process_subtrain(arg_tuple):
    start, end, idx = arg_tuple
    X, y = prep_data(start, end)
    return idx, X, y

num_cores = 8 
all_chunks = []

total_size = len(df_train)
chunk_size = total_size/num_cores

for i in range(8):
    start_idx = int(i * chunk_size)
    end_idx = int(start_idx + chunk_size)
    chunk = (start_idx, end_idx, i)
    all_chunks.append(chunk)
    
all_chunks[0]

pool = Pool()
results = pool.map(process_subtrain, all_chunks)

sorted(results, key=lambda tup: tup[0])

np.concatenate([item[1][1] for item in results], axis=0).shape

In [19]:
# this code is very simple, divide the total size of the df_train into two sets and process it
#X = []
#y = []
all_chunks = []

num_cores = 8 
#def load_all():
total_size = len(df_train)
chunk_size = total_size/num_cores

for i in range(8):
    start_idx = int(i * chunk_size)
    end_idx = int(start_idx + chunk_size)
    chunk = (start_idx, end_idx, i)
    all_chunks.append(chunk)

pool = Pool()
results = pool.map(process_subtrain, all_chunks)    
results = sorted(results, key=lambda tup: tup[0])

X = np.concatenate([item[1] for item in results], axis=0)
y = np.concatenate([item[2] for item in results], axis=0)

#load_all()

#X = np.asarray(X)
#y = np.asarray(y)

100%|██████████| 363/363 [15:52<00:00,  1.27s/it]
Process ForkPoolWorker-13:
Process ForkPoolWorker-10:
Process ForkPoolWorker-15:
Process ForkPoolWorker-14:
Process ForkPoolWorker-16:
Process ForkPoolWorker-11:
Process ForkPoolWorker-9:
Process ForkPoolWorker-12:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/cwademo123/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/cwademo123/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/cwademo123/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/cwademo123/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/cwademo1

In [45]:
# The X shape here is very important. It is also important undertand a little how a LSTM works
# X.shape[0] is the number of id_measuremts contained in train data
# X.shape[1] is the number of chunks resultant of the transformation, each of this date enters in the LSTM serialized
# This way the LSTM can understand the position of a data relative with other and activate a signal that needs
# a serie of inputs in a specifc order.
# X.shape[3] is the number of features multiplied by the number of phases (3)
print(X.shape, y.shape)

(2904, 160, 87) (2904,)


In [46]:
# save data into file, a numpy specific format
np.save("X_2.npy",X)
np.save("y_2.npy",y)

In [74]:
X = np.load("./X_2.npy")
y = np.load("./y_2.npy")

X[np.isnan(X)] = 0

X = min_max_transf(X, np.min(X), np.max(X), range_needed=(-1,1))

In [14]:
# This is NN LSTM Model creation
def model_lstm(input_shape):
    # The shape was explained above, must have this order
    inp = Input(shape=(input_shape[1], input_shape[2],))
    
    
    # This is the LSTM layer
    # Bidirecional implies that the 160 chunks are calculated in both ways, 0 to 159 and 159 to zero
    # although it appear that just 0 to 159 way matter, I have tested with and without, and tha later worked best
    # 128 and 64 are the number of cells used, too many can overfit and too few can underfit
    x = Bidirectional(LSTM(128, return_sequences=True))(inp)
    # The second LSTM can give more fire power to the model, but can overfit it too
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    # Attention is a new tecnology that can be applyed to a Recurrent NN to give more meanings to a signal found in the middle
    # of the data, it helps more in longs chains of data. A normal RNN give all the responsibility of detect the signal
    # to the last cell. Google RNN Attention for more information :)
    x = Attention(input_shape[1])(x)
    # A intermediate full connected (Dense) can help to deal with nonlinears outputs
    x = Dense(64)(x)
    
    x = LeakyReLU(alpha=0.1)(x)
    # A binnary classification as this must finish with shape (1,)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    # Pay attention in the addition of matthews_correlation metric in the compilation, it is a success factor key
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[matthews_correlation])
    
    return model

In [48]:
# Here is where the training happens

# First, create a set of indexes of the 5 folds
splits = list(StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=2019).split(X, y))
preds_val = []
y_val = []
# Then, iteract with each fold
# If you dont know, enumerate(['a', 'b', 'c']) returns [(0, 'a'), (1, 'b'), (2, 'c')]
for idx, (train_idx, val_idx) in enumerate(splits):
    K.clear_session() # I dont know what it do, but I imagine that it "clear session" :)
    print("Beginning fold {}".format(idx+1))
    # use the indexes to extract the folds in the train and validation data
    train_X, train_y, val_X, val_y = X[train_idx], y[train_idx], X[val_idx], y[val_idx]
    # instantiate the model for this fold
    model = model_lstm(train_X.shape)
    # This checkpoint helps to avoid overfitting. It just save the weights of the model if it delivered an
    # validation matthews_correlation greater than the last one.
    ckpt = ModelCheckpoint('weights_{}.h5'.format(idx), save_best_only=True, save_weights_only=True, verbose=1, monitor='val_matthews_correlation', mode='max')
    # Train, train, train
    model.fit(train_X, train_y, batch_size=128, epochs=50, validation_data=[val_X, val_y], callbacks=[ckpt])
    # loads the best weights saved by the checkpoint
    model.load_weights('weights_{}.h5'.format(idx))
    # Add the predictions of the validation to the list preds_val
    preds_val.append(model.predict(val_X, batch_size=512))
    # and the val true y
    y_val.append(val_y)

# concatenates all and prints the shape    
preds_val = np.concatenate(preds_val)[...,0]
y_val = np.concatenate(y_val)
preds_val.shape, y_val.shape

Beginning fold 1
Train on 2322 samples, validate on 582 samples
Epoch 1/50
2322/2322 [==============================] - 29s 12ms/step - loss: 0.2620 - matthews_correlation: -0.0043 - val_loss: 0.1858 - val_matthews_correlation: 0.0000e+00

Epoch 00001: val_matthews_correlation improved from -inf to 0.00000, saving model to weights_0.h5
Epoch 2/50
2322/2322 [==============================] - 21s 9ms/step - loss: 0.1598 - matthews_correlation: 0.0273 - val_loss: 0.1297 - val_matthews_correlation: 0.0000e+00

Epoch 00002: val_matthews_correlation did not improve from 0.00000
Epoch 3/50
2322/2322 [==============================] - 21s 9ms/step - loss: 0.1333 - matthews_correlation: 0.3086 - val_loss: 0.1406 - val_matthews_correlation: -0.0039

Epoch 00003: val_matthews_correlation did not improve from 0.00000
Epoch 4/50
2322/2322 [==============================] - 21s 9ms/step - loss: 0.1260 - matthews_correlation: 0.3577 - val_loss: 0.1191 - val_matthews_correlation: 0.6323

Epoch 00004: 

2323/2323 [==============================] - 21s 9ms/step - loss: 0.0815 - matthews_correlation: 0.7554 - val_loss: 0.1154 - val_matthews_correlation: 0.5513

Epoch 00017: val_matthews_correlation did not improve from 0.62283
Epoch 18/50
2323/2323 [==============================] - 21s 9ms/step - loss: 0.0722 - matthews_correlation: 0.7837 - val_loss: 0.1471 - val_matthews_correlation: 0.4815

Epoch 00018: val_matthews_correlation did not improve from 0.62283
Epoch 19/50
2323/2323 [==============================] - 21s 9ms/step - loss: 0.0670 - matthews_correlation: 0.7746 - val_loss: 0.1197 - val_matthews_correlation: 0.5898

Epoch 00019: val_matthews_correlation did not improve from 0.62283
Epoch 20/50
2323/2323 [==============================] - 21s 9ms/step - loss: 0.0645 - matthews_correlation: 0.7824 - val_loss: 0.1412 - val_matthews_correlation: 0.5556

Epoch 00020: val_matthews_correlation did not improve from 0.62283
Epoch 21/50
2323/2323 [==============================] - 21s

Beginning fold 3
Train on 2323 samples, validate on 581 samples
Epoch 1/50
2323/2323 [==============================] - 25s 11ms/step - loss: 0.3035 - matthews_correlation: 0.0000e+00 - val_loss: 0.2284 - val_matthews_correlation: 0.0000e+00

Epoch 00001: val_matthews_correlation improved from -inf to 0.00000, saving model to weights_2.h5
Epoch 2/50
2323/2323 [==============================] - 22s 10ms/step - loss: 0.2220 - matthews_correlation: 0.0000e+00 - val_loss: 0.2057 - val_matthews_correlation: 0.0000e+00

Epoch 00002: val_matthews_correlation did not improve from 0.00000
Epoch 3/50
2323/2323 [==============================] - 21s 9ms/step - loss: 0.1745 - matthews_correlation: -4.5439e-04 - val_loss: 0.1718 - val_matthews_correlation: 0.3448

Epoch 00003: val_matthews_correlation improved from 0.00000 to 0.34475, saving model to weights_2.h5
Epoch 4/50
2323/2323 [==============================] - 21s 9ms/step - loss: 0.1387 - matthews_correlation: 0.2617 - val_loss: 0.1532 - v

2323/2323 [==============================] - 21s 9ms/step - loss: 0.0418 - matthews_correlation: 0.8711 - val_loss: 0.1915 - val_matthews_correlation: 0.6130

Epoch 00034: val_matthews_correlation did not improve from 0.66461
Epoch 35/50
2323/2323 [==============================] - 22s 9ms/step - loss: 0.0658 - matthews_correlation: 0.7942 - val_loss: 0.1051 - val_matthews_correlation: 0.6548

Epoch 00035: val_matthews_correlation did not improve from 0.66461
Epoch 36/50
2323/2323 [==============================] - 21s 9ms/step - loss: 0.0393 - matthews_correlation: 0.8810 - val_loss: 0.1253 - val_matthews_correlation: 0.5482

Epoch 00036: val_matthews_correlation did not improve from 0.66461
Epoch 37/50
2323/2323 [==============================] - 21s 9ms/step - loss: 0.0571 - matthews_correlation: 0.8076 - val_loss: 0.1189 - val_matthews_correlation: 0.5610

Epoch 00037: val_matthews_correlation did not improve from 0.66461
Epoch 38/50
2323/2323 [==============================] - 21s

2324/2324 [==============================] - 21s 9ms/step - loss: 0.0778 - matthews_correlation: 0.7205 - val_loss: 0.1663 - val_matthews_correlation: 0.5289

Epoch 00017: val_matthews_correlation did not improve from 0.65765
Epoch 18/50
2324/2324 [==============================] - 21s 9ms/step - loss: 0.0913 - matthews_correlation: 0.7041 - val_loss: 0.1143 - val_matthews_correlation: 0.6246

Epoch 00018: val_matthews_correlation did not improve from 0.65765
Epoch 19/50
2324/2324 [==============================] - 21s 9ms/step - loss: 0.0760 - matthews_correlation: 0.7786 - val_loss: 0.1151 - val_matthews_correlation: 0.4874

Epoch 00019: val_matthews_correlation did not improve from 0.65765
Epoch 20/50
2324/2324 [==============================] - 21s 9ms/step - loss: 0.0781 - matthews_correlation: 0.7194 - val_loss: 0.1317 - val_matthews_correlation: 0.6502

Epoch 00020: val_matthews_correlation did not improve from 0.65765
Epoch 21/50
2324/2324 [==============================] - 21s


Epoch 00034: val_matthews_correlation did not improve from 0.62756
Epoch 35/50
2324/2324 [==============================] - 22s 9ms/step - loss: 0.0597 - matthews_correlation: 0.7970 - val_loss: 0.1408 - val_matthews_correlation: 0.5593

Epoch 00035: val_matthews_correlation did not improve from 0.62756
Epoch 36/50
2324/2324 [==============================] - 21s 9ms/step - loss: 0.0409 - matthews_correlation: 0.8827 - val_loss: 0.1788 - val_matthews_correlation: 0.5457

Epoch 00036: val_matthews_correlation did not improve from 0.62756
Epoch 37/50
2324/2324 [==============================] - 21s 9ms/step - loss: 0.0363 - matthews_correlation: 0.9010 - val_loss: 0.1823 - val_matthews_correlation: 0.5923

Epoch 00037: val_matthews_correlation did not improve from 0.62756
Epoch 38/50
2324/2324 [==============================] - 21s 9ms/step - loss: 0.0359 - matthews_correlation: 0.8973 - val_loss: 0.1828 - val_matthews_correlation: 0.5709

Epoch 00038: val_matthews_correlation did not i

((2904,), (2904,))

In [49]:
np.save('./tmp/y_val_tmp_4.npy', y_val)
np.save('./tmp/preds_val_tmp_4.npy', preds_val)

In [50]:
# It is the official metric used in this competition
# below is the declaration of a function used inside the keras model, calculation with K (keras backend / thensorflow)
def matthews_correlation(y_true, y_pred):
    '''Calculates the Matthews correlation coefficient measure for quality
    of binary classification problems.
    '''
    
    #y_pred = K.cast(y_pred, np.float)
    y_pred_pos = np.round(np.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = np.round(np.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    tp = np.sum(y_pos * y_pred_pos)
    tn = np.sum(y_neg * y_pred_neg)

    fp = np.sum(y_neg * y_pred_pos)
    fn = np.sum(y_pos * y_pred_neg)

    numerator = (tp * tn - fp * fn)
    denominator = np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    return numerator / (denominator + K.epsilon())

In [51]:
# The output of this kernel must be binary (0 or 1), but the output of the NN Model is float (0 to 1).
# So, find the best threshold to convert float to binary is crucial to the result
# this piece of code is a function that evaluates all the possible thresholds from 0 to 1 by 0.01
def threshold_search(y_true, y_proba):
    best_threshold = 0
    best_score = 0
    for threshold in tqdm([i * 0.01 for i in range(100)]):
        #score = matthews_correlation(y_true, (y_proba > threshold).astype(int))
        #score = K.eval(matthews_correlation(y_true, (y_proba > threshold).astype(int)))
        #score = K.eval(matthews_correlation(y_true.astype(np.float64), (y_proba > threshold).astype(np.float64)))
        score = matthews_correlation(y_true.astype(np.float64), (y_proba > threshold).astype(np.float64))
        if score > best_score:
            best_threshold = threshold
            best_score = score
    search_result = {'threshold': best_threshold, 'matthews_correlation': best_score}
    return search_result

In [52]:
best_threshold = threshold_search(y_val, preds_val)['threshold']

100%|██████████| 100/100 [00:00<00:00, 9742.87it/s]


In [78]:
best_threshold

In [54]:
matthews_correlation(y_val, preds_val)

0.6968210072383328

In [15]:
%%time
# Now load the test data
# This first part is the meta data, not the main data, the measurements
meta_test = pd.read_csv('../input/metadata_test.csv')
df_test = meta_test.set_index(['id_measurement', 'phase'])

CPU times: user 12.8 ms, sys: 5.14 ms, total: 17.9 ms
Wall time: 33.6 ms


In [16]:
meta_test = meta_test.set_index(['signal_id'])
meta_test.head()

,id_measurement,phase
signal_id,,
8712,2904,0
8713,2904,1
8714,2904,2
8715,2905,0
8716,2905,1


In [17]:
np.ceil(20337/8)

2543.0

In [18]:
2543*8

20344

In [82]:
all_chunks

[(0, 2543, 0),
 (2543, 5086, 1),
 (5086, 7629, 2),
 (7629, 10172, 3),
 (10172, 12715, 4),
 (12715, 15258, 5),
 (15258, 17801, 6),
 (17801, 20337, 7)]

In [84]:
2543/3

847.6666666666666

In [83]:
df_test.index.levels[0].unique()

Int64Index([2904, 2905, 2906, 2907, 2908, 2909, 2910, 2911, 2912, 2913,
            ...
            9673, 9674, 9675, 9676, 9677, 9678, 9679, 9680, 9681, 9682],
           dtype='int64', name='id_measurement', length=6779)

In [37]:
praq_test = pq.read_pandas('../input/test.parquet', columns=[str(i) for i in range(8712, 11255)]).to_pandas()

In [38]:
praq_test.head()

,8712,8713,8714,8715,8716,8717,8718,8719,8720,8721,...,11245,11246,11247,11248,11249,11250,11251,11252,11253,11254
0,15,-20,3,11,1,-22,-15,-11,20,-7,...,-3,-12,-9,-13,19,-1,20,-15,16,-18
1,15,-20,3,11,1,-21,-16,-12,19,-7,...,-5,-22,-10,-13,20,-2,20,-15,16,-19
2,14,-21,2,14,2,-20,-14,-9,20,-9,...,-2,-12,-9,-13,20,-1,21,-14,17,-18
3,14,-21,2,13,1,-22,-14,-8,22,-7,...,-3,-15,-9,-13,20,0,22,-14,17,-17
4,15,-20,3,11,2,-22,-12,-6,24,-8,...,-4,-20,-8,-12,20,-1,22,-13,16,-18


In [88]:
2543+8712

11255

In [105]:
df_test.index.levels[0].unique()[int(17801/3):int(20337/3)]

Int64Index([8837, 8838, 8839, 8840, 8841, 8842, 8843, 8844, 8845, 8846,
            ...
            9673, 9674, 9675, 9676, 9677, 9678, 9679, 9680, 9681, 9682],
           dtype='int64', name='id_measurement', length=846)

In [106]:
df_test.tail()

signal_id
id_measurement phase           
9681           1          29044
               2          29045
9682           0          29046
               1          29047
               2          29048

In [ ]:
len(df_train)

In [111]:
all_chunks

[(0, 2543, 0),
 (2543, 5086, 1),
 (5086, 7629, 2),
 (7629, 10172, 3),
 (10172, 12715, 4),
 (12715, 15258, 5),
 (15258, 17801, 6),
 (17801, 20337, 7)]

In [113]:
df_test.loc[3750].loc[2]

signal_id    11252
Name: 2, dtype: int64

In [112]:
df_test.index.levels[0].unique()[int(0/3):int(2543/3)]

Int64Index([2904, 2905, 2906, 2907, 2908, 2909, 2910, 2911, 2912, 2913,
            ...
            3741, 3742, 3743, 3744, 3745, 3746, 3747, 3748, 3749, 3750],
           dtype='int64', name='id_measurement', length=847)

In [114]:
df_test.index.levels[0].unique()[int(2543/3):int(5086/3)]

Int64Index([3751, 3752, 3753, 3754, 3755, 3756, 3757, 3758, 3759, 3760,
            ...
            4589, 4590, 4591, 4592, 4593, 4594, 4595, 4596, 4597, 4598],
           dtype='int64', name='id_measurement', length=848)

In [34]:
df_test.index.levels[0].unique()[int(1272/3):int(2544/3)]

Int64Index([3328, 3329, 3330, 3331, 3332, 3333, 3334, 3335, 3336, 3337,
            ...
            3742, 3743, 3744, 3745, 3746, 3747, 3748, 3749, 3750, 3751],
           dtype='int64', name='id_measurement', length=424)

In [48]:
df_test.loc[3328].loc[0][0]

9984

In [49]:
str(df_test.loc[3327].loc[2][0])

'9983'

In [36]:
1272+8712

9984

In [50]:
# this function take a piece of data and convert using transform_ts(), but it does to each of the 3 phases
# if we would try to do in one time, could exceed the RAM Memmory
def prep_data_test(start, end):
    # load a piece of data from file
    praq_test = pq.read_pandas('../input/test.parquet', columns=[str(i) for i in range(start+8712, end+8712)]).to_pandas()
    X = []

    # using tdqm to evaluate processing time
    # takes each index from df_train and iteract it from start to end
    # it is divided by 3 because for each id_measurement there are 3 id_signal, and the start/end parameters are id_signal
    for id_measurement in tqdm(df_test.index.levels[0].unique()[int(start/3):int(end/3)]):
        X_signal = []
        # for each phase of the signal
        for phase in [0,1,2]:
            # extract from df_train both signal_id and target to compose the new data sets
            signal_id = df_test.loc[id_measurement].loc[phase][0]
            # but just append the target one time, to not triplicate it
            
            # extract and transform data into sets of features
            X_signal.append(transform_ts(praq_test[str(signal_id)]))
        # concatenate all the 3 phases in one matrix
        X_signal = np.concatenate(X_signal, axis=1)
        # add the data to X
        X.append(X_signal)
    X = np.asarray(X)
    return X

In [20]:
def process_subtest(arg_tuple):
    start, end, idx = arg_tuple
    X = prep_data_test(start, end)
    return idx, X

In [60]:
all_chunks = []

num_cores = 16 
#def load_all():
total_size = len(meta_test)
chunk_size = np.ceil(total_size/num_cores)
#train_size = len(df_train)

for i in range(16):
    if i != 15:
        start_idx = int(i * chunk_size)
        end_idx = int(start_idx + chunk_size)
        #chunk = (start_idx+train_size, end_idx+train_size, i)
        chunk = (start_idx, end_idx, i)
        all_chunks.append(chunk)
    else:
        start_idx = int(i * chunk_size)
        end_idx = int(total_size)
        #chunk = (start_idx+train_size, end_idx+train_size, i)
        chunk = (start_idx, end_idx, i)
        all_chunks.append(chunk)
        

In [61]:
import gc
gc.collect()

75

In [62]:
pool = Pool()
results_1 = pool.map(process_subtest, all_chunks[0:8])    
results_1 = sorted(results_1, key=lambda tup: tup[0])

100%|██████████| 424/424 [17:50<00:00,  1.25s/it]


In [63]:
results_2 = pool.map(process_subtest, all_chunks[8:16])    
results_2 = sorted(results_2, key=lambda tup: tup[0])

In [67]:
results = results_1 + results_2
X_test = np.concatenate([item[1] for item in results], axis=0)

In [69]:
np.save("X_test_2.npy",X_test)

In [ ]:
all_chunks = []

num_cores = 8 
#def load_all():
total_size = len(meta_test)
chunk_size = np.ceil(total_size/num_cores)

for i in range(8):
    start_idx = int(i * chunk_size)
    end_idx = int(start_idx + chunk_size)
    chunk = (start_idx, end_idx, i)
    all_chunks.append(chunk)

pool = Pool()
results = pool.map(process_subtrain, all_chunks)    
results = sorted(results, key=lambda tup: tup[0])

X = np.concatenate([item[1] for item in results], axis=0)
y = np.concatenate([item[2] for item in results], axis=0)

In [57]:
meta_test.index[0]

8712

In [58]:
first_sig = meta_test.index[0]
n_parts = 10
max_line = len(meta_test)
part_size = int(max_line / n_parts)
last_part = max_line % n_parts
print(first_sig, n_parts, max_line, part_size, last_part, n_parts * part_size + last_part)

8712 10 20337 2033 7 20337


In [59]:
# Here we create a list of lists with start index and end index for each of the 10 parts and one for the last partial part
start_end = [[x, x+part_size] for x in range(first_sig, max_line + first_sig, part_size)]
start_end = start_end[:-1] + [[start_end[-1][0], start_end[-1][0] + last_part]]
print(start_end)

[[8712, 10745], [10745, 12778], [12778, 14811], [14811, 16844], [16844, 18877], [18877, 20910], [20910, 22943], [22943, 24976], [24976, 27009], [27009, 29042], [29042, 29049]]


In [ ]:
%%time
# First we daclarete a series of parameters to initiate the loading of the main data
# it is too large, it is impossible to load in one time, so we are doing it in dividing in 10 parts
first_sig = meta_test.index[0]
n_parts = 10
max_line = len(meta_test)
part_size = int(max_line / n_parts)
last_part = max_line % n_parts
print(first_sig, n_parts, max_line, part_size, last_part, n_parts * part_size + last_part)
# Here we create a list of lists with start index and end index for each of the 10 parts and one for the last partial part
start_end = [[x, x+part_size] for x in range(first_sig, max_line + first_sig, part_size)]
start_end = start_end[:-1] + [[start_end[-1][0], start_end[-1][0] + last_part]]
print(start_end)
X_test = []
# now, very like we did above with the train data, we convert the test data part by part
# transforming the 3 phases 800000 measurement in matrix (160,57)
for start, end in start_end:
    subset_test = pq.read_pandas('../input/test.parquet', columns=[str(i) for i in range(start, end)]).to_pandas()
    for i in tqdm(subset_test.columns):
        id_measurement, phase = meta_test.loc[int(i)]
        subset_test_col = subset_test[i]
        subset_trans = transform_ts(subset_test_col)
        X_test.append([i, id_measurement, phase, subset_trans])

In [ ]:
X_test_input = np.asarray([np.concatenate([X_test[i][3],X_test[i+1][3], X_test[i+2][3]], axis=1) for i in range(0,len(X_test), 3)])
np.save("X_test.npy",X_test_input)
X_test_input.shape

In [ ]:
X_test_input = np.load('./X_test.npy')

In [71]:
submission = pd.read_csv('../input/sample_submission.csv')
print(len(submission))
submission.head()

20337


,signal_id,target
0,8712,0
1,8713,0
2,8714,0
3,8715,0
4,8716,0


In [75]:
splits = list(StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=2019).split(X, y))

for idx, (train_idx, val_idx) in enumerate(splits):
    K.clear_session() # I dont know what it do, but I imagine that it "clear session" :)
    print("Beginning fold {}".format(idx+1))
    # use the indexes to extract the folds in the train and validation data
    train_X, train_y, val_X, val_y = X[train_idx], y[train_idx], X[val_idx], y[val_idx]
    # instantiate the model for this fold
    model = model_lstm(train_X.shape)

Beginning fold 1
Beginning fold 2
Beginning fold 3
Beginning fold 4
Beginning fold 5


In [76]:
preds_test = []
for i in range(N_SPLITS):
    model.load_weights('weights_{}.h5'.format(i))
    pred = model.predict(X_test, batch_size=300, verbose=1)
    pred_3 = []
    for pred_scalar in pred:
        for i in range(3):
            pred_3.append(pred_scalar)
    preds_test.append(pred_3)


6779/6779 [==============================] - 16s 2ms/step


In [79]:
preds_test = (np.squeeze(np.mean(preds_test, axis=0)) > best_threshold).astype(np.int)
preds_test.shape

(20337,)

In [80]:
submission['target'] = preds_test
submission.to_csv('../output/submission_18.csv', index=False)
submission.head()

,signal_id,target
0,8712,0
1,8713,0
2,8714,0
3,8715,0
4,8716,0


In [ ]:
lstm_preds = np.squeeze(np.mean(preds_test, axis=0))
np.save("./lstm_preds.npy", lstm_preds)